In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
def RotX(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[1, 1] = ct
    R[1, 2] = -st
    R[2, 1] = st
    R[2, 2] = ct
    return R

def RotY(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 2] = st
    R[2, 0] = -st
    R[2, 2] = ct
    return R

def RotZ(theta):
    ct = np.cos(theta)
    st = np.sin(theta)
    R = np.eye(3, 3)
    R[0, 0] = ct
    R[0, 1] = -st
    R[1, 0] = st
    R[1, 1] = ct
    return R

In [ ]:
# Euler Angles

def eulerZYX_to_rotation_matrix(z, y, x):
    return RotZ(z) @ RotY(y) @ RotX(x)

def eulerXYZ_to_rotation_matrix(x, y, z):
    return RotX(x) @ RotY(y) @ RotZ(z)

# Both of them should output identity
print(eulerZYX_to_rotation_matrix(0., 0., 0.))
print(eulerXYZ_to_rotation_matrix(0., 0., 0.))

print("========================")

# These 2 should output the same
print(eulerZYX_to_rotation_matrix(np.pi, 0., 0.))
print(eulerXYZ_to_rotation_matrix(0., 0., np.pi))

print("========================")

# These 2 should output the same
print(eulerZYX_to_rotation_matrix(0., np.pi, 0.))
print(eulerXYZ_to_rotation_matrix(0., np.pi, 0.))

print("========================")

# These 2 should output something different
print(eulerZYX_to_rotation_matrix(0., np.pi, 0.2))
print(eulerXYZ_to_rotation_matrix(0.2, np.pi, 0.))

In [ ]:
# Angle Axis
def hat(vec):
    v = vec.reshape((3,))
    return np.array([
        [0., -v[2], v[1]],
        [v[2], 0., -v[0]],
        [-v[1], v[0], 0.]
    ])

def exp_rotation(p):
    phi = p.reshape((3, 1))
    theta = np.linalg.norm(phi)
    if theta < 1e-12:
        return np.eye(3, 3)
    a = phi / theta
    return np.eye(3) * np.cos(theta) + (1. - np.cos(theta)) * a @ a.T + np.sin(theta) * hat(a)

def log_rotation(R):
    # We do not account for all special cases!
    theta = np.arccos(max(-1., min(1., (np.trace(R) - 1.) / 2.)))
    if theta < 1e-12:
        return np.zeros((3, 1))
    mat = R - R.T
    r = np.array([mat[2, 1], mat[0, 2], mat[1, 0]]).reshape((3, 1))
    return theta / (2. * np.sin(theta)) * r

# Lets' try a few things
R = eulerZYX_to_rotation_matrix(1., 0., 0.)

print(R)
print("========================")
logR = log_rotation(R)
print(logR.T)
print("========================")
print(exp_rotation(logR))

In [ ]:
# Quaternions

def quat_to_rotation_matrix(q):
    q0 = q[0, 0]
    q1 = q[1, 0]
    q2 = q[2, 0]
    q3 = q[3, 0]

    # First row of the rotation matrix
    r00 = 2. * (q0 * q0 + q1 * q1) - 1.
    r01 = 2. * (q1 * q2 - q0 * q3)
    r02 = 2. * (q1 * q3 + q0 * q2)

    # Second row of the rotation matrix
    r10 = 2. * (q1 * q2 + q0 * q3)
    r11 = 2. * (q0 * q0 + q2 * q2) - 1.
    r12 = 2. * (q2 * q3 - q0 * q1)

    # Third row of the rotation matrix
    r20 = 2. * (q1 * q3 - q0 * q2)
    r21 = 2. * (q2 * q3 + q0 * q1)
    r22 = 2. * (q0 * q0 + q3 * q3) - 1.

    # 3x3 rotation matrix
    R = np.array([[r00, r01, r02],
                  [r10, r11, r12],
                  [r20, r21, r22]])

    return R

# Lets' try a few things
theta = 1.
axis = np.array([[0., 0., 1.]]).T

# quaternion
q = np.block([[np.cos(theta/2.)], [axis * np.sin(theta/2)]])

# angle axis
logR = theta * axis

print(q.T)
print("========================")
print(logR.T)
print("========================")
print(quat_to_rotation_matrix(q))
print("========================")
print(exp_rotation(logR))

In [ ]:
# Angular Velocities
def angular_velocity_to_deriv_rotation(omega_w, R):
    return hat(omega_w) @ R

def angular_velocity_local_to_deriv_rotation(omega_b, R):
    return R @ hat(omega_b)

# Some testing
R = eulerZYX_to_rotation_matrix(1., 0., 0.)

omega_b = np.array([[1., 0., 0.]]).T

# Integrate naively for a few steps
dt = 0.1
R1 = np.copy(R)
R2 = np.copy(R)

for i in range(5):
    omega_w = R1 @ omega_b # rotate local omega to world frame
    R1d = angular_velocity_to_deriv_rotation(omega_w, R1)
    R2d = angular_velocity_local_to_deriv_rotation(omega_b, R2)

    R1 = R1 + R1d * dt # Euler integration!
    R2 = R2 + R2d * dt # Euler integration!

# These two should be very similar
print(R1)
print("========================")
print(R2)
print("========================")
print(np.linalg.norm(R1-R2))

In [ ]:
def quat_multiply(qa, qb):
    sa = qa[0]
    sb = qb[0]
    va = qa[1:]
    vb = qb[1:]

    qf = np.zeros((4, 1))
    qf[0] = sa*sb - va.T @ vb
    qf[1:] = sa*vb + sb*va + np.cross(va, vb, axis=0)

    return qf

def angular_velocity_local_to_quat(omega_b, q):
    ### TO-DO: Implement q_dot from angular velocity
    q_omega = np.block([[0.], [omega_b]])
    return 0.5 * quat_multiply(q, q_omega)
    ### END of TO-DO

# Some testing
theta = 1.
axis = np.array([[0., 0., 1.]]).T

# quaternion
q = np.block([[np.cos(theta/2.)], [axis * np.sin(theta/2)]])

# rotation matrix
R = quat_to_rotation_matrix(q)

omega_b = np.array([[1., 0., 0.]]).T

# Integrate naively for a few steps
dt = 0.001
Ri = np.copy(R)
qi = np.copy(q)

for i in range(20):
    qid = angular_velocity_local_to_quat(omega_b, qi)
    Rid = angular_velocity_local_to_deriv_rotation(omega_b, Ri)

    qi = qi + qid * dt # Euler integration!
    # renormalize quaternion
    qi = qi / np.linalg.norm(qi)
    Ri = Ri + Rid * dt # Euler integration!
Rq = quat_to_rotation_matrix(qi)
# These two should be very similar
print(Rq)
print("========================")
print(Ri)
print("========================")
print(np.linalg.norm(Rq-Ri))